In [1]:
import pandas as pd
import numpy as np
from autofeat import FeatureSelector
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import IsolationForest

In [2]:
raw_dataset_x = pd.read_csv('X_train.csv',index_col=['id'])
X = raw_dataset_x.copy()
X.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,14168.823171,10514.380717,3316.149698,94230.695124,102.386606,92.677127,11108.748199,10866.505510,10837.622093,10.227734,...,NaN,12352.094085,846.014651,105.132144,102.112809,2090.004260,2.691845,1234.374109,1000.784475,9285.751272
1.0,17757.037554,NaN,4101.016273,92959.527633,NaN,99.855168,10013.959449,10826.607494,10076.101597,11.436970,...,NaN,16198.071494,776.084467,106.385590,103.472030,2474.051881,2.287976,NaN,1012.626705,11750.284764
2.0,14226.656663,11029.642499,NaN,124055.600561,100.542483,92.860892,NaN,10492.342868,NaN,10.810076,...,10329.704431,13976.063780,737.040332,103.671234,109.458246,2656.083281,2.843706,888.353607,1048.810385,9553.922728
3.0,8766.012436,7384.202998,2147.308418,100157.719990,104.855061,101.929026,10050.049932,10499.521099,10525.030989,10.092109,...,10008.251395,6212.127347,329.044233,105.084488,104.858546,1097.785204,2.732257,927.752967,1048.357330,NaN
4.0,13801.016418,13269.493652,3408.316953,92048.527786,103.759758,95.789235,9667.353978,10750.783106,10618.800750,12.006773,...,10095.782015,13772.061493,NaN,NaN,100.369834,2693.053231,2.702908,1471.354073,1071.284484,9423.533063


In [3]:
raw_dataset_y = pd.read_csv('y_train.csv',index_col=['id'])
y = raw_dataset_y.copy()
y.head()

,y
id,
0.0,74.0
1.0,51.0
2.0,70.0
3.0,52.0
4.0,85.0


In [4]:
missing_values = X.isna().sum()
missing_values

x0       91
x1       84
x2       88
x3       91
x4       74
       ... 
x827    101
x828     95
x829     92
x830     80
x831     96
Length: 832, dtype: int64

In [5]:
for col in X.columns[X.nunique(dropna = False) <= 100]:
    print(col, " ", X[col].unique())

x104   [ 0. nan]
x129   [ 0. nan]
x489   [ 0. nan]
x530   [ 0. nan]


In [6]:
X.drop(columns = X.columns[X.nunique(dropna = False) <= 100], inplace = True)

In [7]:
#X = X.fillna(X.mean())
imputer = KNNImputer(n_neighbors=2)
X = pd.DataFrame(imputer.fit_transform(X,y))

In [8]:
X.describe().transpose().sort_values('min')

,count,mean,std,min,25%,50%,75%,max
171,1212.0,-1.836984e+22,2.502749e+22,-2.937933e+23,-2.861816e+22,-1.384906e+22,-3.738343e+21,7.002915e+22
661,1212.0,4.624869e+22,4.016835e+22,-6.392632e+22,2.722623e+22,3.942073e+22,6.067928e+22,9.935455e+23
592,1212.0,-1.796460e+14,1.260767e+14,-8.756492e+14,-2.348957e+14,-1.486501e+14,-9.590108e+13,2.926746e+14
130,1212.0,-1.385710e+06,2.094810e+05,-2.819624e+06,-1.499255e+06,-1.364324e+06,-1.279261e+06,-5.738037e+05
372,1212.0,-5.413822e+05,7.268332e+04,-1.002561e+06,-5.818485e+05,-5.337267e+05,-5.051311e+05,-1.607672e+05
...,...,...,...,...,...,...,...,...
465,1212.0,1.051675e+06,2.746376e+04,1.000249e+06,1.029032e+06,1.053533e+06,1.072575e+06,1.099984e+06
419,1212.0,1.049964e+06,2.730667e+04,1.000304e+06,1.027581e+06,1.050468e+06,1.072518e+06,1.099971e+06
633,1212.0,1.050106e+06,2.788380e+04,1.000329e+06,1.026186e+06,1.051442e+06,1.072637e+06,1.099951e+06
329,1212.0,1.048944e+06,2.837349e+04,1.000372e+06,1.024670e+06,1.050083e+06,1.070840e+06,1.099887e+06


In [9]:
model=IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.2),random_state=np.random.RandomState(42))
model.fit(X)
X['anomaly'] = model.predict(X)
y['anomaly'] = X['anomaly']
X['anomaly'].value_counts()
X = X[X.anomaly != -1]
y = y[y.anomaly != -1]

X.drop('anomaly',axis=1,inplace=True)
y.drop('anomaly',axis=1,inplace=True)


In [10]:
# selector = VarianceThreshold()
# x_train_tmp = selector.fit_transform(X,y)
selector = SelectKBest(f_regression, k=100)
selector.fit(X, y)
cols = selector.get_support(indices=True)
dataset_selected_x = X.iloc[:,cols]
dataset_selected_x


/home/francescog/ETH/ETH_AML_projects_2022/task_1/venv/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/francescog/ETH/ETH_AML_projects_2022/task_1/venv/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:306: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


,2,23,27,29,40,69,87,106,112,114,...,722,727,730,741,744,762,765,773,797,813
0,3316.149698,2.823225,8945.065857,1665.073010,-442319.204404,299199.981189,9496.005953,2.194544,14114.997688,12802.203605,...,4791.107191,9197.051884,2.512959,2.561577,2.886191,2.110701,2.333374,2.555374,2.305030,2.588126
1,4101.016273,2.677011,10407.086185,2115.015631,-564055.951889,383995.232158,11212.089706,2.261742,16292.423778,1921.015297,...,6688.960032,12853.015438,2.476245,2.695044,2.723722,2.057154,2.052895,2.575797,2.384952,2.417485
2,3289.867648,2.702694,8063.053692,1985.045559,-451105.028773,304547.452877,9277.004195,2.375093,11899.476849,2167.802432,...,4045.043798,9056.035902,2.574194,2.534789,2.652784,2.134853,2.334419,2.525839,2.477187,2.476064
4,3408.316953,2.647249,8426.068899,2700.034646,-478206.543121,324191.752487,8629.004273,2.239014,13843.961128,6424.918043,...,5785.680263,9830.068890,2.211228,2.540963,2.597508,2.045608,2.152986,2.431158,2.263235,2.316878
6,3928.789039,2.754380,11373.050453,1556.069043,-543773.353065,367732.899574,8510.506873,2.120987,17004.907988,23824.758898,...,6234.587981,10904.088779,2.398018,2.625697,2.735732,2.138059,2.264274,2.480709,2.284575,2.280422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,3437.591663,2.700417,6677.045448,1615.056922,-374393.087183,254412.666774,8510.506873,2.101966,12306.124090,3356.707347,...,4494.686554,7721.096118,2.113575,2.563214,2.711036,1.908956,2.136680,2.496045,2.126682,2.095136
1208,3442.619145,2.540225,9423.047150,2251.094278,-521774.326758,353025.264014,10020.022198,2.240806,12829.056866,4141.182106,...,6124.743194,9781.056572,2.259954,2.474671,2.672402,1.924956,2.304004,2.460164,2.306389,2.255477
1209,3580.894003,2.873919,8443.094334,1779.050779,-489924.690574,330968.805912,8685.026317,2.444754,14427.867597,2433.848973,...,5069.867722,11616.067779,2.560814,2.783308,2.746683,2.368872,2.369319,2.421252,2.518431,2.485791
1210,3159.167789,2.690725,8753.038127,2117.011836,-473393.482968,317582.071472,8517.089186,2.427602,13121.156248,4703.596560,...,4590.892701,10654.021613,2.385572,2.570085,2.569565,2.234898,2.329367,2.578617,2.493086,2.560172


In [11]:
#new_X.describe()

In [12]:
#x_train_tmp = pd.DataFrame()

In [13]:
# fsel = FeatureSelector(
#         problem_type='regression',
#         featsel_runs=6,
#         keep=None,
#         n_jobs=3,
#         verbose=1)

# new_X = fsel.fit_transform(x_train_tmp, y)

In [14]:
#new_X.columns.values

In [15]:


# for j in range(832):
#         if j in accepted:
#           print (j)
#         else:
#           del dataset_selected_x[str(j)]
#dataset_selected_x = dataset_selected_x.iloc[:,cols]
# train_stats = dataset_selected_x.describe()
# train_stats = train_stats.transpose()
#dataset_selected_x

In [16]:
dataset_selected_x.to_csv('X_preprocessed.csv',index_label="id")
y.to_csv('y_preprocessed.csv',index_label="id")